In [1]:
import pandas as pd
import numpy as np

# Deck

In [2]:
class Deck():
    values = ["A", 2, 3, 4, 5, 6, 7, 8, 9, 10, "J", "Q", "K"]
    def __init__(self, num_decks=1, values=values):
        # Multiply values by number of suits
        # Then by number of decks
        self.num_decks = num_decks
        self.deck = values * 4 * self.num_decks
        
    def __str__(self):
        return "{} decks, {} cards left".format(self.num_decks, len(self.deck))
    
    def shuffle(self):
        np.random.shuffle(self.deck)
    
    def deal(self, hand=None):
        if hand == None:
            return self.deck.pop(0)
        else:
            hand.append(self.deck.pop(0))

In [4]:
test = Deck()
print (test)
print (test.deck)
print (test.deal())
print (test.deck)
test.shuffle()
print (test.deck)

1 decks, 52 cards left
['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
A
[2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K', 'A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'Q', 'K']
['Q', 5, 10, 10, 2, 'K', 'Q', 7, 4, 2, 6, 'A', 'K', 9, 8, 8, 5, 9, 5, 3, 'K', 'J', 10, 8, 3, 'K', 3, 4, 2, 9, 5, 'A', 7, 4, 6, 7, 3, 'A', 9, 2, 6, 10, 4, 'J', 8, 7, 'J', 'Q', 'J', 6, 'Q']


# Calculate points

In [5]:
def calculate_points(hand):
    points = 0
    num_ace = 0
    
    # Deal with aces
    while "A" in hand:
        hand.remove("A")
        num_ace += 1
    
    # Deal with the rest of the hand
    for i in hand:
        try:
            # If it's a number, add it to points
            points += i
        except:
            # If it's not a number, it's a 10
            points += 10
            
    # deal with the aces
    for i in range(num_ace):
        if points + 11 <= 21:
            points += 11
        else:
            points += 1
    return points

In [6]:
calculate_points([5, 5, "A"])

21

# Simulate game

In [15]:
data_dictionary = pd.read_csv("data_dictionary.csv")
data_dictionary

,Feature,Type,Description
0,num_decks,Int,Number of decks used
1,dealer_open,Int,The card we can see
2,dealer_initial,Int,The dealer's starting points
3,dealer_hit,Binary,"1 - dealer hit, 0 - dealer did not hit"
4,dealer_num_hits,Int,Number of times the dealer hit
5,dealer_final,Int,The dealer's final points
6,dealer_busts,Binary,"1 - dealer busts, 0 - dealer did not bust"
7,player_card_one,Int,Player's first card
8,player_card_two,Int,Player's second card
9,player_inital,Int,The player's starting points


In [12]:
def sim_game(num_decks=1, strategy=0):
    game_deck = Deck(num_decks=num_decks)
    game_deck.shuffle()

    dealer_hand = []
    player_hand = []

    # deal 2 cards each
    for i in range(2):
        game_deck.deal(player_hand)
        game_deck.deal(dealer_hand)

    # get their initial points
    # copy so that our hand is not changed
    player_initial = calculate_points(player_hand.copy())
    dealer_initial = calculate_points(dealer_hand.copy())

    # get dealer open card
    dealer_open = dealer_hand[0]

    # get cards from player
    player_card_one = player_hand[0]
    player_card_two = player_hand[1]

    # Change them to numerics
    if dealer_open in ["J", "Q", "K"]:
        dealer_open = 10
    elif dealer_open == "A":
        dealer_open = 1

    if player_card_one in ["J", "Q", "K"]:
        player_card_one = 10
    elif player_card_one == "A":
        player_card_one = 1

    if player_card_two in ["J", "Q", "K"]:
        player_card_two = 10
    elif player_card_two == "A":
        player_card_two = 1

    dealer_hit = 0
    dealer_num_hits = 0
    player_hit = 0
    player_num_hits = 0
    player_busts = 0
    dealer_busts = 0
    dealer_final = calculate_points(dealer_hand.copy())
    player_final = calculate_points(player_hand.copy())


    # if anyone got a blackjack, the game should end
    if player_initial != 21 and dealer_initial != 21:
        # if neither of them got a blackjack game continues
        # the player goes first

        # If player <= 11, hit
        while calculate_points(player_hand.copy()) <= 11:
            player_hit = 1
            player_num_hits += 1
            game_deck.deal(player_hand)

        # If strategy is random, randomize hit for 18 and below
        if strategy == 0:
            while calculate_points(player_hand.copy()) <= 18:
                if np.random.random() < 0.5:
                    player_hit = 1
                    player_num_hits += 1
                    game_deck.deal(player_hand)
        # If strategy is recommended, stand on 17 and above
        else:
            if dealer_open <= 6:
                while calculate_points(player_hand.copy()) <= 16:
                    player_hit = 1
                    player_num_hits += 1
                    game_deck.deal(player_hand)

        # update player's final and busts
        player_final = calculate_points(player_hand.copy())
        player_busts = player_final > 21
        # dealer's turn

        # If player didn't bust
        if not player_busts:
            # If dealer < 17, hit
            while calculate_points(dealer_hand.copy()) < 17:
                dealer_hit = 1
                dealer_num_hits += 1
                game_deck.deal(dealer_hand)

            # If dealer has a soft 17, hit
            if calculate_points(dealer_hand.copy()) == 17:
                if "A" in dealer_hand:
                    dealer_copy = dealer_hand.copy()
                    dealer_copy.remove("A")
                    if calculate_points(dealer_copy.copy()) == 6:
                        dealer_hit = 1
                        dealer_num_hits += 1
                        game_deck.deal(dealer_hand)

        # update dealer's final and busts
        dealer_final = calculate_points(dealer_hand.copy())
        dealer_busts = dealer_final > 21

    player_loses = 0
    draw = 0
    player_wins = 0

    # Check who wins
    if player_initial == 21 and dealer_initial != 21:
        player_wins = 1
    elif dealer_initial == 21 and player_initial != 21:
        player_loses = 1
    elif player_busts:
        player_loses = 1
    elif dealer_busts:
        player_wins = 1
    elif player_final > dealer_final:
        player_wins = 1
    elif player_final < dealer_final:
        player_loses = 1
    elif player_final == dealer_final:
        draw = 1

    # Change hands to strings
    dealer_hand_str = ",".join([str(i) for i in dealer_hand])
    player_hand_str = ",".join([str(i) for i in player_hand])
    
    return np.array([num_decks, dealer_open, dealer_initial, 
                     dealer_hit, dealer_num_hits, dealer_final, 
                     int(dealer_busts), player_card_one, 
                     player_card_two, player_initial, player_hit, 
                     player_num_hits, player_final, int(player_busts), 
                     player_loses, draw, player_wins, strategy, 
                     dealer_hand_str, player_hand_str])

In [16]:
pd.Series(sim_game(), index=data_dictionary.Feature.values)

num_decks                1
dealer_open              1
dealer_initial          18
dealer_hit               0
dealer_num_hits          0
dealer_final            18
dealer_busts             0
player_card_one          7
player_card_two          6
player_inital           13
player_hit               1
player_num_hits          2
player_final            26
player_busts             1
player_loses             1
draw                     0
player_wins              0
strategy                 0
dealer_hand            A,7
player_hand        7,6,5,8
dtype: object

# Generate a dataframe

## Requirements

- run sim_game() for some number of times and save the output to a df

In [17]:
data_dictionary.Feature.values

array(['num_decks', 'dealer_open', 'dealer_initial', 'dealer_hit',
       'dealer_num_hits', 'dealer_final', 'dealer_busts',
       'player_card_one', 'player_card_two', 'player_inital', 'player_hit',
       'player_num_hits', 'player_final', 'player_busts', 'player_loses',
       'draw', 'player_wins', 'strategy', 'dealer_hand', 'player_hand'], dtype=object)

In [18]:
def gen_data(num_decks=1, df_size=5000, strategy=0):
    return np.array([sim_game(num_decks=num_decks, strategy=strategy) for _ in range(df_size)])

In [19]:
def gen_df(data):
    tmp = pd.DataFrame(data, columns=data_dictionary.Feature.values)
    tmp[tmp.columns.values[:-2]] = tmp[tmp.columns.values[:-2]].astype(int)
    return tmp

In [20]:
ran = gen_df(gen_data(num_decks=4))

In [21]:
rec = gen_df(gen_data(num_decks=4, strategy=1))

In [22]:
df = pd.concat([ran,rec])

In [25]:
# Combine old files
old_df = pd.read_csv("blackjack.csv")
df = pd.concat([old_df, df])

In [26]:
# save to file
df.to_csv("blackjack.csv", index=False)

# Testing our code

In [27]:
df = pd.read_csv("blackjack.csv")

In [28]:
df.shape

(60000, 22)

In [29]:
# When player_loses = 1, draw = 0, player_wins = 0
print (df.draw[df.player_loses == 1].value_counts())
print (df.player_wins[df.player_loses == 1].value_counts())

0    32755
Name: draw, dtype: int64
0    32755
Name: player_wins, dtype: int64


In [30]:
# When player_loses = 0, draw = 1, player_wins = 0
print (df.player_loses[df.draw == 1].value_counts())
print (df.player_wins[df.draw == 1].value_counts())

0    4348
Name: player_loses, dtype: int64
0    4348
Name: player_wins, dtype: int64


In [31]:
# When player_loses = 0, draw = 0, player_wins = 1
print (df.player_loses[df.player_wins == 1].value_counts())
print (df.draw[df.player_wins == 1].value_counts())

0    22897
Name: player_loses, dtype: int64
0    22897
Name: draw, dtype: int64


In [32]:
# When dealer_busts = 1, player_loses = 0, draw = 0, player_wins = 1
print (df.player_loses[df.dealer_busts == 1].value_counts())
print (df.draw[df.dealer_busts == 1].value_counts())
print (df.player_wins[df.dealer_busts == 1].value_counts())

0    10739
Name: player_loses, dtype: int64
0    10739
Name: draw, dtype: int64
1    10739
Name: player_wins, dtype: int64


In [33]:
# When player_busts = 1, player_loses = 1, draw = 0, player_wins = 0, dealer_hit = 0, dealer_busts = 0
print (df.player_loses[df.player_busts == 1].value_counts())
print (df.draw[df.player_busts == 1].value_counts())
print (df.player_wins[df.player_busts == 1].value_counts())
print (df.dealer_hit[df.player_busts == 1].value_counts())
print (df.dealer_busts[df.player_busts == 1].value_counts())

1    17766
Name: player_loses, dtype: int64
0    17766
Name: draw, dtype: int64
0    17766
Name: player_wins, dtype: int64
0    17766
Name: dealer_hit, dtype: int64
0    17766
Name: dealer_busts, dtype: int64


In [34]:
# When dealer_busts = 0 and player_busts = 0, if player_final > dealer final, player_loses = 0, draw = 0, player_wins = 1
sub = df[(df.player_busts == 0) & (df.dealer_busts == 0)]
print (sub.player_loses[sub.player_final > sub.dealer_final].value_counts())
print (sub.draw[sub.player_final > sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final > sub.dealer_final].value_counts())

0    12158
Name: player_loses, dtype: int64
0    12158
Name: draw, dtype: int64
1    12158
Name: player_wins, dtype: int64


In [35]:
# When dealer_busts = 0 and player_busts = 0, if player_final = dealer final, player_loses = 0, draw = 1, player_wins = 0
print (sub.player_loses[sub.player_final == sub.dealer_final].value_counts())
print (sub.draw[sub.player_final == sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final == sub.dealer_final].value_counts())

0    4348
Name: player_loses, dtype: int64
1    4348
Name: draw, dtype: int64
0    4348
Name: player_wins, dtype: int64


In [36]:
# When dealer_busts = 0 and player_busts = 0, if player_final < dealer final, player_loses = 1, draw = 0, player_wins = 0
print (sub.player_loses[sub.player_final < sub.dealer_final].value_counts())
print (sub.draw[sub.player_final < sub.dealer_final].value_counts())
print (sub.player_wins[sub.player_final < sub.dealer_final].value_counts())

1    14989
Name: player_loses, dtype: int64
0    14989
Name: draw, dtype: int64
0    14989
Name: player_wins, dtype: int64


In [37]:
# When player_final <= 21, player_busts == 0
print (df.player_busts[df.player_final <= 21].value_counts())

0    42234
Name: player_busts, dtype: int64


In [38]:
# When dealer_final <= 21, dealer_busts == 0
print (df.dealer_busts[df.dealer_final <= 21].value_counts())

0    49261
Name: dealer_busts, dtype: int64


In [39]:
# When player_hit == 1, player_num_hits >= 1
print (df.player_num_hits[df.player_hit == 1].value_counts())

1    21225
2     9925
3     2487
4      366
5       47
6        5
Name: player_num_hits, dtype: int64


In [40]:
# When dealer_hit == 1, dealer_num_hits >= 1
print (df.dealer_num_hits[df.dealer_hit == 1].value_counts())

1    16893
2     6444
3     1206
4      113
5       12
Name: dealer_num_hits, dtype: int64


In [41]:
df.dealer_hit.value_counts()

0    35332
1    24668
Name: dealer_hit, dtype: int64

In [42]:
df.dealer_num_hits.value_counts()

0    35332
1    16893
2     6444
3     1206
4      113
5       12
Name: dealer_num_hits, dtype: int64

In [43]:
df.player_hit.value_counts()

1    34055
0    25945
Name: player_hit, dtype: int64

In [44]:
df.player_num_hits.value_counts()

0    25945
1    21225
2     9925
3     2487
4      366
5       47
6        5
Name: player_num_hits, dtype: int64

In [67]:
df[["dealer_final", "player_final"]][df.strategy == 0].describe()

,dealer_final,player_final
count,30000.000000,30000.000000
mean,17.203167,21.825800
std,4.535142,3.231264
min,4.000000,4.000000
25%,14.000000,20.000000
50%,18.000000,21.000000
75%,20.000000,24.000000
max,26.000000,28.000000


In [68]:
df[["dealer_final", "player_final"]][df.strategy == 1].describe()

,dealer_final,player_final
count,30000.000000,30000.000000
mean,18.982200,17.951100
std,4.034182,3.512557
min,4.000000,4.000000
25%,17.000000,15.000000
50%,19.000000,18.000000
75%,21.000000,20.000000
max,26.000000,26.000000
